In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import os
import sys
import statsmodels.api as sm
%matplotlib inline
# 重回帰    =>   https://to-kei.net/python/data-analysis/linearregression/
     
        
df=pd.read_csv('df.csv')
df=df.drop('Unnamed: 0', axis=1)
df.head()

,buyer_hasso_average,buyer_kaitsuke_seiko_average,rating_good,point,coupon,safety_tesuryo,popularity_boost,teikei_cd,minyuka_tag,login,label
0,2.120000,99.030000,0,0,0,12,0,0,0,30,0
1,3.850000,99.060000,0,0,0,0,0,0,0,135,0
2,5.877647,93.954117,12,0,0,34,0,0,4,85,0
3,2.570000,93.410000,25,0,0,25,0,0,0,25,0
4,1.130000,99.430000,15,0,0,15,0,0,0,30,0


In [14]:
# u1とu2に分ける
df1=df.query("label==0")
df2=df.query("label==1")

df1=df1.drop('label', axis=1)
df2=df2.drop('label', axis=1)
df2=df2.reset_index().drop('index', axis=1)
print(df1.shape)
print(df2.shape)

(463476, 10)
(2025505, 10)


In [13]:
# u1 =>  tag_id(minyuka_tag) が 0以上のケース
df1=df1.query("minyuka_tag>0")
df2=df2.query("minyuka_tag>0")
print(df1.shape)
print(df2.shape)

(250853, 10)
(1169898, 10)


In [15]:
#  u1 => 最も関係性の高い特徴量を探索
target_name=df1.columns
for name in target_name:
    target=pd.DataFrame(df1[name])
    print(target.shape)
    data=df1.drop(name, axis=1)
    print(data.shape)
    clf=LinearRegression()
    clf.fit(data,target)
    print('score:{},name:{}'.format(clf.score(data,target),name))

(463476, 1)
(463476, 9)
score:0.04857584967062989,name:buyer_hasso_average
(463476, 1)
(463476, 9)
score:0.04437247454223525,name:buyer_kaitsuke_seiko_average
(463476, 1)
(463476, 9)
score:0.693179584237263,name:rating_good
(463476, 1)
(463476, 9)
score:0.0394654671947815,name:point
(463476, 1)
(463476, 9)
score:0.5454708380841586,name:coupon
(463476, 1)
(463476, 9)
score:0.45488092798353796,name:safety_tesuryo
(463476, 1)
(463476, 9)
score:0.0030521244598372688,name:popularity_boost
(463476, 1)
(463476, 9)
score:0.2871698598582906,name:teikei_cd
(463476, 1)
(463476, 9)
score:0.6968805971884896,name:minyuka_tag
(463476, 1)
(463476, 9)
score:0.6422706087574331,name:login


In [16]:
# 目的変数
target=pd.DataFrame(df1['minyuka_tag'])
print(target.shape)

# 特徴量
data=df1.drop('minyuka_tag', axis=1)
data = sm.add_constant(data)
print(data.shape)

(463476, 1)
(463476, 10)


In [17]:
# 説明変数の標準化
f1_columns=list(data.columns)

sc = StandardScaler()
dff = sc.fit_transform(data)
dff=pd.DataFrame(dff)
dff.columns=f1_columns
dff.head()

,const,buyer_hasso_average,buyer_kaitsuke_seiko_average,rating_good,point,coupon,safety_tesuryo,popularity_boost,teikei_cd,login
0,0.0,-0.798087,0.607758,-0.171634,-0.028633,-0.030194,-0.010636,-0.049963,-0.125604,0.006339
1,0.0,-0.284062,0.610778,-0.171634,-0.028633,-0.030194,-0.249043,-0.049963,-0.125604,0.473642
2,0.0,0.318402,0.096896,0.049329,-0.028633,-0.030194,0.426446,-0.049963,-0.125604,0.251117
3,0.0,-0.664381,0.042133,0.288705,-0.028633,-0.030194,0.247640,-0.049963,-0.125604,-0.015913
4,0.0,-1.092241,0.648016,0.104569,-0.028633,-0.030194,0.048966,-0.049963,-0.125604,0.006339


In [18]:
# 目的変数の標準化
t_columns=list(target.columns)

t = sc.fit_transform(target)
t=pd.DataFrame(t)
t.columns=t_columns
t.head()

,minyuka_tag
0,-0.192254
1,-0.192254
2,0.290862
3,-0.192254
4,-0.192254


In [19]:
# 結果の詳細を表示
model = sm.OLS(target, data)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            minyuka_tag   R-squared:                       0.697
Model:                            OLS   Adj. R-squared:                  0.697
Method:                 Least Squares   F-statistic:                 1.184e+05
Date:                Wed, 19 Sep 2018   Prob (F-statistic):               0.00
Time:                        12:54:55   Log-Likelihood:            -1.3607e+06
No. Observations:              463476   AIC:                         2.721e+06
Df Residuals:                  463466   BIC:                         2.722e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

In [20]:
# 各特徴量の重要度
clf=LinearRegression()
clf.fit(dff,t)
clf.score(dff,t)
name = list(dff.columns)
conf = pd.DataFrame(clf.coef_)
conf.columns=name
conf=conf.drop('const', axis=1)
conf

,buyer_hasso_average,buyer_kaitsuke_seiko_average,rating_good,point,coupon,safety_tesuryo,popularity_boost,teikei_cd,login
0,0.039667,-0.010731,0.721705,-0.082173,0.402151,0.248743,0.004272,-0.238808,-0.031757


In [21]:
# u2

df=pd.read_csv('df.csv')
df=df.drop('Unnamed: 0', axis=1)
df1=df.query("label==0")
df2=df.query("label==1")

df1=df1.drop('label', axis=1)
df2=df2.drop('label', axis=1)
df2=df2.reset_index().drop('index', axis=1)
print(df1.shape)
print(df2.shape)

(463476, 10)
(2025505, 10)


In [22]:
# rating_goodが１以上のケース
df1=df1.query("rating_good>0")
df2=df2.query("rating_good>0")
print(df1.shape)
print(df2.shape)

(139998, 8)
(167274, 8)


In [22]:
# 最も関係性の高い特徴量を探索
target_name=df2.columns
for name in target_name:
    target=pd.DataFrame(df2[name])
    print(target.shape)
    data=df2.drop(name, axis=1)
    print(data.shape)
    clf=LinearRegression()
    clf.fit(data,target)
    print('score:{},name:{}'.format(clf.score(data,target),name))

(2025505, 1)
(2025505, 9)
score:0.039094092460516316,name:buyer_hasso_average
(2025505, 1)
(2025505, 9)
score:0.032384244888621794,name:buyer_kaitsuke_seiko_average
(2025505, 1)
(2025505, 9)
score:0.8370702607091367,name:rating_good
(2025505, 1)
(2025505, 9)
score:0.08817284557894178,name:point
(2025505, 1)
(2025505, 9)
score:0.1836286963600534,name:coupon
(2025505, 1)
(2025505, 9)
score:0.8295242158434777,name:safety_tesuryo
(2025505, 1)
(2025505, 9)
score:0.008578829758829842,name:popularity_boost
(2025505, 1)
(2025505, 9)
score:0.1442970929656665,name:teikei_cd
(2025505, 1)
(2025505, 9)
score:0.3151313302411344,name:minyuka_tag
(2025505, 1)
(2025505, 9)
score:0.5465267364290278,name:login


In [23]:
# 目的変数
target=pd.DataFrame(df2['rating_good'])
print(target.shape)

# 特徴量
data=df2.drop('rating_good', axis=1)
data = sm.add_constant(data)
print(data.shape)

(2025505, 1)
(2025505, 10)


In [24]:
# 説明変数の標準化
f1_columns=list(data.columns)

sc = StandardScaler()
dff = sc.fit_transform(data)
dff=pd.DataFrame(dff)
dff.columns=f1_columns
dff.head()

,const,buyer_hasso_average,buyer_kaitsuke_seiko_average,point,coupon,safety_tesuryo,popularity_boost,teikei_cd,minyuka_tag,login
0,0.0,-0.169723,0.720454,-0.054038,-0.106047,-0.168931,-0.037009,-0.167875,-0.141509,-0.260152
1,0.0,-0.961355,0.492993,-0.054038,-0.106047,-0.112535,-0.037009,-0.167875,-0.260311,-0.079944
2,0.0,0.363768,-0.108154,-0.054038,-0.106047,-0.150132,-0.037009,-0.167875,-0.260311,-0.290187
3,0.0,-0.628640,-0.137781,-0.054038,-0.106047,-0.150132,-0.037009,-0.072195,-0.260311,-0.290187
4,0.0,-0.548330,-0.007803,-0.054038,-0.106047,-0.097496,-0.037009,-0.167875,-0.260311,-0.255862


In [25]:
# 目的変数の標準化
t_columns=list(target.columns)

t = sc.fit_transform(target)
t=pd.DataFrame(t)
t.columns=t_columns
t.head()

,rating_good
0,-0.131598
1,-0.142909
2,-0.142909
3,-0.112746
4,-0.059962


In [33]:
model = sm.OLS(target, data)
results = model.fit()
 
#結果の詳細を表示
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            rating_good   R-squared:                       0.756
Model:                            OLS   Adj. R-squared:                  0.756
Method:                 Least Squares   F-statistic:                 8.268e+04
Date:                Mon, 10 Sep 2018   Prob (F-statistic):               0.00
Time:                        16:31:45   Log-Likelihood:            -7.7719e+05
No. Observations:              187119   AIC:                         1.554e+06
Df Residuals:                  187111   BIC:                         1.554e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.0062      0.040  

In [34]:
# 各特徴量の重要度
clf=LinearRegression()
clf.fit(dff,t)
clf.score(dff,t)
name = list(dff.columns)
conf = pd.DataFrame(clf.coef_)
conf.columns=name
conf=conf.drop('const', axis=1)
conf

,coupon,point,rating_bad,safety_tesuryo,popularity_boost,teikei_cd,login
0,0.182995,0.102981,-0.026088,0.596717,0.049363,0.077837,0.131544
